# Outline
1. Python and Jupyter Intro
1. Dataset examination
1. Linear Regression and Gradient Descent
1. Logistic Regression

Many thanks and indebted to Daniel Bowring, Chris Mayes, Auralee.

![title](Learn.png)

## 1. Python and Jupyter Intro

In [ ]:
## Load libraries
# Numpy: for scientific computing and arrays
import numpy as np
# Matplotlib: for plotting
import matplotlib.pyplot as plt
# Scipy: For maths, science, engineering
import scipy.stats as stats
# Seaborn: high level plots
import seaborn as sns
# Pandas: data analysis package
import pandas as pd

### Array

In [ ]:
# Numpy Array
a = np.array([1,2,3,4,5])

In [ ]:
# Inspect value 
a

In [ ]:
type(a)

In [ ]:
# Python starts counting at 0!
a[0]

In [ ]:
# Easy array manipulation
a * 2

In [ ]:
# Length
len(a)

In [ ]:
# 2D Array
b = np.array([[1,2],[4,5],[7,8]])

In [ ]:
b

In [ ]:
# Often useful to get shape of matrix
np.shape(b)

In [ ]:
b[1]

In [ ]:
# b[1] is short for
b[1,:]

In [ ]:
# Access column
b[:,1]

In [ ]:
# For loop
for i in a:
    print(i)

### Dictionary

A dictionary is a collection which is unordered, changeable and indexed. In Python dictionaries are written with curly brackets, and they have keys and values.

In [ ]:
thisWorkshop = {
  "name": "2nd ICFA Mini-Workshop on Machine Learning for Charged Particle Accelerators",
  "place": "PSI",
  "year": 2019
}

In [ ]:
thisWorkshop['name']

### Plotting

In [ ]:
plt.plot(a, np.sqrt(a), 'o-');

## 2. Dataset

Beamline with two solenoids (strengths $K_1$, $K_2$), two cavities (strengths $G_1$, $G_2$ phases $\phi_1$, $\phi_2$).

Simulated with beam dynamics code OPAL (https://gitlab.psi.ch/OPAL/Manual-2.0/wikis/home)

Outputs: emittances, energy, beamsize

Many thanks to Auralee, Nicole Neveu and Andreas Adelmann for providing the dataset!

Based on a model of the Argonne Wakefield Accelerator.

Note that the dataset is not the same as a soon to be published paper on accelerator surrogate models.

![title](Beamline.png)

In [ ]:
# Load data
filename = "ANN_7ksliced.pk"
data = np.load(filename)

## Exercise 0: inspect data format

In [ ]:
# How does our data format look like?
# Your code here, see notebook for an example

Data format not so useful for data analysis so we will transform to something more suitable

In [ ]:
# Numpy arrays
dname  = np.array(data[0]['dvarNames']) # input (design) names
oname  = np.array(data[0]['objNames'])  # output names
names  = data[0]['dvarNames'] + data[0]['objNames'] # all names
dval   = np.asarray(data[1]['dvarValues'],dtype=np.float64) # convert string to float
oval   = np.asarray(data[1]['objValues'],dtype=np.float64)  # output values
values = np.concatenate((dval,oval),axis=1) # all values

In [ ]:
dname

In [ ]:
oname

Pandas "DataFrame" : "Two-dimensional size-mutable, potentially heterogeneous tabular data structure with labeled axes"

In [ ]:
df = pd.DataFrame(data    = values,
                  columns = names);

## Exercise 1: Data inspection

In [ ]:
# How does the data look like?
# Do we need to clean the data?
# Which parameter can we throw away?
# Your code here, see notebook for an example

There seem to be no clear outliers.
Output variable 'numParticles' not interesting 

For the rest no cleaning needed (unusual!)

In [ ]:
# throw away 'numParticles' column (hackish way)
dname  = np.array(data[0]['dvarNames']) # input (design) names
oname  = np.array(data[0]['objNames'][:-1])  # output names
names  = data[0]['dvarNames'] + data[0]['objNames'][:-1] # all names
dval   = np.asarray(data[1]['dvarValues'],dtype=np.float64) # convert string to float
oval   = np.asarray(data[1]['objValues'],dtype=np.float64)[:,:-1]  # output values
values = np.concatenate((dval,oval),axis=1) # all values
df     = pd.DataFrame(data    = values,
                      columns = names);

Some patterns seen, let's try to find some more.

In [ ]:
# Correlation Plot
correlations = np.empty((len(dname),len(oname)))
for i in range(len(dname)):
    for j in range(len(oname)):
            #compute correlation
            correlations[i,j],_ = stats.spearmanr(dval[:,i], oval[:,j])

In [ ]:
correlations

In [ ]:
# plot correlations
fig = plt.figure(figsize=((8,6))) # slightly larger
ax = fig.add_subplot(111)
cax = ax.matshow(correlations,cmap='seismic')
fig.colorbar(cax)
plt.xticks(range(len(oname)), oname)
plt.yticks(range(len(dname)), dname);

In [ ]:
# Let's inspect the energy correlation
G1     = dval[:,3]
energy = oval[:,7]
plt.plot(G1,energy,'.');
plt.xlabel(dname[3])
plt.ylabel(oname[7]);

Can we predict energy?

## 3. Linear Regression

![title](LinReg.png)

The math behind linear regression:

$\hat{y}=\theta_0+\theta_1x_1+\cdots+\theta_nx_n = \theta^T\cdot\mathbf{x} = h_\theta(x)$

Mean square error: our cost function

$\mathrm{MSE}(X,h_\theta)=\frac{1}{m}\sum^{m}_{i=1}(\theta^T\cdot x^{(i)}-y^{(i)})^2$

The normal equation is an analytic solution that minimizes $\theta$:

$\hat{\theta}=(X^T\cdot X)^{-1}\cdot X^T\cdot y$

In [ ]:
# The best-fit according to mean square error has an analytic solution: the normal equation.
# RMS error also ok, but more computationally involved.
G1_b = np.c_[np.ones((len(G1),1)),G1] # 1's and G1
theta_best = np.linalg.inv(G1_b.T.dot(G1_b)).dot(G1_b.T).dot(energy)
print(theta_best)

In [ ]:
plt.plot(G1,energy,'.');
plt.xlabel(dname[3])
plt.ylabel(oname[7]);
plt.plot(G1, theta_best[0]+theta_best[1]*G1, 'r', linewidth=2, label='mean square fit')
plt.legend();

In [ ]:
# Error on our model
error1 = energy - (theta_best[0]+theta_best[1]*G1)
rel_error1 = error1 / energy

In [ ]:
plt.hist(rel_error1);
plt.xlabel("relative error");

Not bad, but we can do better

## Exercise 2: Add second parameter

In [ ]:
# Your code here, see notebook for an example
# What is the error on the prediction?

## Sklearn

In [ ]:
# Now do all this again, using sklearn. Will be easier to code this time.
# scikit-learn: Machine Learning in Python
import sklearn
from sklearn.linear_model import LinearRegression

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(dval, energy) # Let's use all input information this time

# The fit method imparts attributes intercept_ and coef_, which are just what you think they are.
lin_reg.intercept_, lin_reg.coef_

In [ ]:
# prediction from our model
prediction = lin_reg.predict(dval)
rel_error_all = (prediction - energy) / energy

In [ ]:
rel_error_all

In [ ]:
plt.hist(rel_error1, label='1 parameter');
plt.hist(rel_error_all, label = 'all parameters');
plt.xlabel("relative error")
plt.legend();

## Gradient Descent

For more complicated functions, there is no analytical solution for the best fit.
Now we'll investigate the 'machine learning' way of finding the best fit.

Define a cost fuction. At feature coordinates, calculate cost function gradient. "Go down."

Good policy to make sure all features have similar scale. Small partial derivatives will make search times longer.

$\frac{\partial}{\partial\theta_j}\mathrm{MSE}(\theta)=\frac{2}{m}\sum_{i=1}^m(\theta^T\cdot x^{(i)}-y^{(i)})x^{(i)}_j$

$\theta_{i+1} = \theta_i-\eta\nabla_\theta\mathrm{MSE}(\theta)$

$\eta=$ training rate, determining size of step in min search.

In [ ]:
def bgd(X, y, eta, n_iteration, theta0):
    # batch gradient descent: use whole training set to compute gradient at every iteration
    # X = observed data / "features"
    # y = target 
    # eta = training rate
    # n_iteration = define total number of steps in search
    # theta0 = starting guess
    batch_thetas = np.empty((n_iteration,2)) # vector of fit parameters found by *batch* gradient descent (see below)
    X_b   = np.c_[np.ones((len(y),1)),X] # add 1 vector for offset
    y     = y.reshape(-1,1)              # reshape if not done
    m     = len(y)                       # vector length
    theta = theta0
    
    for iteration in range(n_iteration):
        gradients = 2/m * X_b.T.dot(X_b.dot(theta)-y)
        theta = theta - eta*gradients
        batch_thetas[iteration] = theta.T # populate batch_thetas vector with search steps
    return theta, batch_thetas

In [ ]:
# Normalisation array: data between [0,1]
def norm(x):
    return (x-min(x))/(max(x)-min(x))

In [ ]:
# Normalise our parameters (you can also try to see what happens without)
G1_norm = norm(G1) # Normalise
E_norm  = norm(energy)

In [ ]:
# Calculate new solution for normalised values
lin_reg.fit(G1_norm.reshape(-1,1), E_norm)
# The fit method imparts attributes intercept_ and coef_, which are just what you think they are.
theta0_best = np.array([lin_reg.intercept_, lin_reg.coef_[0]])

In [ ]:
%%time
# How long does this gradient search take?

# How does training rate affect fit quality, for a constant n_iteration?
# Make a vector of etas.
etas = [0.1, 0.01, 0.001]
n_iteration = 1000
batch_thetas = np.empty((len(etas), n_iteration, 2))
thetas       = np.empty((len(etas), 2))
for i, eta in enumerate(etas):            # enumerate for loop: both index and value 
    theta0 = np.random.randn(2,1)         # pick a random starting point in feature-space
    # Fit results from len(etas) different gradient descent searches
    (thetas[i,0], thetas[i,1]), batch_thetas[i] = bgd(G1_norm, E_norm, eta, n_iteration, theta0)

In [ ]:
fig1, ax1 = plt.subplots()
ax1.scatter(G1_norm, E_norm) # same as default plot
colors = ['orange','cyan','magenta']
for i, eta in enumerate(etas):
    # Plot each search result
    ax1.plot(G1_norm, thetas[i,0]+G1_norm*thetas[i,1], label='$\eta=$'+str(eta), color=colors[i])
ax1.plot(G1_norm, theta0_best[0]+theta0_best[1]*G1_norm, 'r:', linewidth=2, label='normal equation')
plt.legend()
plt.xlabel(dname[3])
plt.ylabel(oname[7]);
plt.title('Different training rates');

In [ ]:
# Visualize batch gradient descent: plot coefficients in theta-space for each step in BGD.
for i, eta in enumerate(etas):
    plt.plot(batch_thetas[i][:,0], batch_thetas[i][:,1], '.-', label = '$\eta=$'+str(eta))
plt.plot([theta0_best[0]], [theta0_best[1]], 'k*', label='best fit')
plt.xlabel("theta_0")
plt.ylabel("theta_1")
plt.legend();

In [ ]:
# Visualise interactively
%matplotlib inline
import time
import pylab as pl
from IPython import display

fig2, ax2 = plt.subplots()
ax2.set_xlabel('theta0')
ax2.set_ylabel('theta1')
ax2.scatter(G1_norm, E_norm)
ax2.plot(G1_norm, theta0_best[0]+theta0_best[1]*G1_norm, 'r-.', linewidth=2, label='normal equation')
ax2.legend()
for th in batch_thetas[1][::100]: # every 100 iterations
    # watch the fit converge as we get closer to cost minimum
    ax2.plot(G1_norm, th[0]+G1_norm*th[1])
    display.clear_output(wait=True)
    display.display(pl.gcf())
    time.sleep(0.1)
display.clear_output(wait=True)

Stochastic gradient descent
-------

At every step, pick a random instance of training set and only compute gradient on that instance

This is a random walk. Cost function won't relate monotonically to iteration step number.

Con = noise, even at cost minimum

Pro = more resistant to local fake-me-out minima

Compromise also possible: Large steps at first to jump out of local minima, and smaller steps once you start converging on the true minimum. *Simulated annealing*, based on a *learning schedule* which tunes $\eta$ per iteration.

In [ ]:
# One of those field-specific conventions.
# Iterations clumped into m groups: "epochs".
n_epochs = 20

# learning schedule hyperparameters
# set up a learning schedule so eta varies by iteration step
t0, t1 = 5, 20
def learning_schedule(t):
    # Learning schedule 
    return t0 / (t+t1)

In [ ]:
def sgd(X, y, n_epochs, theta0):
    sgd_thetas = np.empty((n_epochs*len(y),2)) # vector of fit parameters found by *batch* gradient descent (see below)
    X_b   = np.c_[np.ones((len(y),1)),X] # add 1 vector for offset
    y     = y.reshape(-1,1)              # reshape if not done
    m     = len(y)                       # vector length
    theta = theta0
    for epoch in range(n_epochs):
        for i in range(m):
            random_index = np.random.randint(m)
            xi = X_b[random_index:random_index+1]
            yi = y  [random_index:random_index+1]
            gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
            eta = learning_schedule(epoch*m + i)
            theta = theta - eta*gradients    
            sgd_thetas[i + epoch*m] = theta.T
    return theta, sgd_thetas

In [ ]:
%%time
theta0 = np.random.randn(2,1)         # pick a random starting point in feature-space
theta, sgd_thetas = sgd(G1_norm, E_norm, n_epochs, theta0)

In [ ]:
print(theta)
print('Difference relative to normal equation:', theta[0]-theta0_best[0], theta[1]-theta0_best[1])
# Compare: 20 steps for stochastic gradient descent, relative to 1000 for batch GD.

Mini batch gradient descent
------------

A compromise between computing the true gradient and the gradient at a single example is to compute the gradient against more than one training example (called a "mini-batch") at each step.

This can perform significantly better than "true" stochastic gradient descent described, because the code can make use of vectorization libraries rather than computing each step separately. It may also result in smoother convergence, as the gradient computed at each step is averaged over more training examples.

In [ ]:
def mbgd(X, y, n_epochs, mb, theta0):
    # mb  minibatch size
    m = len(y)               # vector length
    n_mb = m // mb           # number of minibatches (// integer division) 
    sgd_thetas = np.empty((n_epochs*n_mb,2)) # vector of fit parameters found by *batch* gradient descent (see below)
    X_b   = np.c_[np.ones((len(y),1)),X] # add 1 vector for offset
    y     = y.reshape(-1,1)              # reshape if not done
    theta = theta0
    for epoch in range(n_epochs):
        for i in range(n_mb):
            random_index = np.random.randint(m - mb)
            xi = X_b[random_index:random_index+mb]
            yi = y  [random_index:random_index+mb]
            gradients = 2 * xi.T.dot(xi.dot(theta) - yi)/mb
            eta = learning_schedule(epoch*m + i)
            theta = theta - eta*gradients    
            sgd_thetas[i + epoch * n_mb] = theta.T
    return theta, sgd_thetas

In [ ]:
%%time
mini_batch_size=30
n_epochs = 50
theta0 = np.random.randn(2,1)         # pick a random starting point in feature-space
theta, mb_thetas = mbgd(G1_norm, E_norm, n_epochs, mini_batch_size, theta0)

In [ ]:
# We can look at progress in theta-space for each technique.
fig4, ax4 = plt.subplots(figsize=(15,9))
bd_thetas = batch_thetas[0] # 0: best learning rate of 0.01
ax4.plot(sgd_thetas[:,0],sgd_thetas[:,1],   '.-', label='stochastic gradient descent')
ax4.plot(mb_thetas[:,0], mb_thetas[:,1],    '.-', label='minibatch gradient descent')
ax4.plot(bd_thetas[:,0], bd_thetas[:,1], '.-', label='batch gradient descent')
ax4.set_xlabel('theta_0')
ax4.set_ylabel('theta_1')
ax4.plot(theta0_best[0], theta0_best[1], 'ro', label='true optimum')

ax4.legend(loc='lower right');

In [ ]:
# Curious about error per iteration for each technique.
def getRMS(x, y, x0, y0):
    return ((x-x0)**2+(y-y0)**2)**0.5

In [ ]:
sgd_rms = getRMS(sgd_thetas[:,0],sgd_thetas[:,1],theta0_best[0],theta0_best[1])
mb_rms  = getRMS(mb_thetas [:,0],mb_thetas [:,1],theta0_best[0],theta0_best[1])
bd_rms  = getRMS(bd_thetas [:,0],bd_thetas [:,1],theta0_best[0],theta0_best[1])

In [ ]:
# Plot rms error vs number of steps
plt.figure(figsize=((10,5)))
plt.subplot(121)
plt.semilogx(bd_rms,  label='batch')
plt.semilogx(sgd_rms, label='stochastic')
plt.semilogx(mb_rms,  label='minibatch')
plt.subplot(122)
plt.loglog(bd_rms,  label='batch')
plt.loglog(sgd_rms, label='stochastic')
plt.loglog(mb_rms,  label='minibatch')
plt.legend();

### Sklearn

Now do it with sklearn

https://scikit-learn.org/stable/modules/sgd.html#regression
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor

In [ ]:
from sklearn.linear_model import SGDRegressor # more info with ?SGDRegressor

In [ ]:
%%time
sgd_reg = SGDRegressor(max_iter=100, penalty=None, eta0=0.1, tol=1e-3)
# default learning schedule
# no regularization --> penalty=None
# tol: tolerance of final solution
# Faster so no need to slice!
G1_norm = norm(G1) # Normalise
E_norm  = norm(energy)
G1_norm = G1_norm.reshape(-1, 1) # reshape 1-d to 2-d array
sgd_reg.fit(G1_norm,E_norm); # reshape into 2-d array, 

In [ ]:
sgd_reg.intercept_, sgd_reg.coef_

In [ ]:
print('Difference relative to normal equation:', sgd_reg.intercept_ - theta0_best[0], sgd_reg.coef_ - theta0_best[1])

In [ ]:
predictions_norm = sgd_reg.predict(G1_norm)

In [ ]:
plt.hist(predictions_norm - E_norm);

In [ ]:
# Unnormalise
def unnorm(x, minx, maxx):
    return (x*(maxx-minx) + minx)

In [ ]:
predict = unnorm(predictions_norm, min(energy), max(energy))

In [ ]:
predict

In [ ]:
rel_error = (predict-energy)/energy
plt.hist(rel_error)
plt.xlabel("relative error");

## Exercise 3: Predict $\epsilon_x$ using Gradient Descent

In [ ]:
# Your code here, see notebook for an example

# 4. Logistic Regression

$\sigma(x)=\frac{1}{1+e^{-k(x-x_0)}}$

$x_0$ can be thought of as a bias: how positive/negative does x have to be before the neuron fires?

Action of neural network looks something like this:

$a^{(1)}=\sigma(\mathbf{W}a^{(0)}+b)$

The logistic function can be used as a binary classifier. Computes a weighted sum of input features, but the output is the logistic of that sum. 

$\hat{p}=\sigma(\theta^T\cdot x)$

Model estimates the probability $\hat{p}$ that an instance $x$ belongs to "positive class". This is the basis for predictions under this method.

$\hat{y} =\left\{\begin{array}{c}0,\;\hat{p}<0.5\\1,\;\hat{p}\ge0.5\end{array}\right.$

Training based on a cost function kinda like

$c(\theta)=\left\{\begin{array}{c}-\log(\hat{p}),\quad y=1\\ -\log(1-\hat{p}),\quad y=0\end{array}\right.$

with high cost for probabilities close to zero. No analytic solution to the minimum of this cost function, but the function is convex so it's reasonable to try GD on it.

In [ ]:
# For a change we use pandas dataframe
df.hist('emit_x')
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0));
df.hist('emit_y')
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0));

In [ ]:
# Define good machines as having small final emittance
goodMachines = [(df['emit_x'] < 4e-6)] and [(df['emit_y'] < 4e-6)]

In [ ]:
# Count of the good machines
np.count_nonzero(goodMachines)

In [ ]:
plt.scatter(df['K2'], goodMachines);

## Predict good machines based on input

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
classifier = np.array(goodMachines).ravel() # ravel to make 1d array

In [ ]:
np.shape(classifier)

In [ ]:
K2 = df['K2'].values.reshape(-1,1)

In [ ]:
log_reg = LogisticRegression() # more info with ?LogisticRegression
log_reg.fit(K2,classifier) 

In [ ]:
predictions = log_reg.predict(K2)

In [ ]:
predictions

In [ ]:
# Accuracy (how many are correctly predicted?)
log_reg.score(K2, classifier)

In [ ]:
confidence_scores = log_reg.decision_function(K2)

In [ ]:
plt.hist(confidence_scores);

In [ ]:
# More metrics
from sklearn.metrics import recall_score     # true positives / (true pos. + false neg.)
from sklearn.metrics import precision_score  # true positives / (true pos. + false pos.)

In [ ]:
# How many good machines are well predicted?
recall_score(classifier, predictions)

In [ ]:
# How many predicted machines are actually good?
precision_score(classifier, predictions)

In [ ]:
# ROC curve (Receiver Operating Characteristic)
# False positive rate versus true positive rate for different thresholds 
# (usually 0.5) but can be chosen
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(classifier, confidence_scores, pos_label=True)
roc_auc = auc(fpr, tpr) # area under curve

In [ ]:
plt.figure()
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right");

## Exercise 4: Use all input variables

In [ ]:
# Your code here
# How does the accuracy change with more input?

In [ ]:
![title](LearnLearn.png)